# Starexec Data Extraction

#### Imports

In [6]:
import pandas as pd
import os
import cProfile
import re
import sys
from pathlib import Path
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

starexec_outputs = "data/starexec_outputs"
job_index = 5201
problem_folder = starexec_outputs + f"/Job{str(job_index)}_output/"

### StarExec csv output

In [8]:
df_se_csv = pd.read_csv(problem_folder + f"Job{str(job_index)}_info.csv")
df_se_csv
# df_se_csv.sort_values("wallclock time").reset_index(drop=True)[["cpu time","wallclock time"]].plot(style=".")
# plt.legend()

,pair id,benchmark,benchmark id,solver,solver id,configuration,configuration id,status,cpu time,wallclock time,memory usage,result,expected,SZSResult,SZSOutput,SZSStatus
0,131481248,jannis_gehring/TPTP v9.0.0/Problems/TOP/TOP008...,4193021,2.1.0,3788,PyRes_rd_002,400704,timeout (cpu),5.04,5.21475,328008.0,ResourceOut,Satisfiable,UNK-Non,Non,UNK
1,131481249,jannis_gehring/TPTP v9.0.0/Problems/TOP/TOP008...,4193021,2.1.0,3788,PyRes_rd_003,400707,timeout (cpu),5.04,5.22980,330052.0,ResourceOut,Satisfiable,UNK-Non,Non,UNK
2,131481250,jannis_gehring/TPTP v9.0.0/Problems/TOP/TOP008...,4193021,2.1.0,3788,PyRes_rd_005,400705,timeout (cpu),5.08,5.25208,328004.0,--,Satisfiable,UNK-Non,Non,UNK
3,131481251,jannis_gehring/TPTP v9.0.0/Problems/TOP/TOP008...,4193021,2.1.0,3788,PyRes_rd_008,400701,timeout (cpu),5.06,5.22886,329032.0,--,Satisfiable,UNK-Non,Non,UNK
4,131481252,jannis_gehring/TPTP v9.0.0/Problems/TOP/TOP008...,4193021,2.1.0,3788,PyRes_rd_013,400703,timeout (cpu),5.04,5.22451,326984.0,ResourceOut,Satisfiable,UNK-Non,Non,UNK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33323,131514571,jannis_gehring/TPTP v9.0.0/Problems/MED/MED009...,4178902,2.1.0,3788,PyRes_rd_008,400701,timeout (cpu),5.04,5.23872,301432.0,ResourceOut,Theorem,UNK-Non,Non,UNK
33324,131514572,jannis_gehring/TPTP v9.0.0/Problems/MED/MED009...,4178902,2.1.0,3788,PyRes_rd_013,400703,timeout (cpu),5.02,5.19173,301660.0,ResourceOut,Theorem,UNK-Non,Non,UNK
33325,131514573,jannis_gehring/TPTP v9.0.0/Problems/MED/MED009...,4178902,2.1.0,3788,PyRes_rd_021,400708,timeout (cpu),5.01,5.19103,301156.0,ResourceOut,Theorem,UNK-Non,Non,UNK
33326,131514574,jannis_gehring/TPTP v9.0.0/Problems/MED/MED009...,4178902,2.1.0,3788,PyRes_rd_033,400706,timeout (cpu),5.04,5.21630,301532.0,ResourceOut,Theorem,UNK-Non,Non,UNK


### StarExec individual job files

In [9]:
load_from_file = False
store = True
if load_from_file:
    df_se_indi = pd.read_csv(problem_folder + f"df_se_indi_{str(job_index)}.csv", index_col=0)
else:
    print(f"Searching in {problem_folder}")

    # Get file texts
    file_dir_names = [
        subdir+"/"+files[0]
        for (subdir, dirs, files) in os.walk(problem_folder + "jannis_gehring")
        if len(dirs)==0
    ]
    texts = []
    for full_file_name in tqdm(file_dir_names):
        if not os.path.isfile(full_file_name):
            texts.append(None)
            continue
        with open(full_file_name) as full_file_name:
            s_mp_runs = full_file_name.read()
            texts.append(s_mp_runs)
    df_se_indi = pd.DataFrame({
        "file_dir_name": file_dir_names,
        "text": texts,
    })

    # Search through problems with regex
    # todo: fix: this regex patterns doesnt allow for lines starting with 79.84/10.64
    new_column_specs = [
        ("problem",                         r"(?<=Problem    : )(\S+)(?= : )",               str),
        ("python_version",                  r"(?<=Python )(\S+)",                            str),
        ("cpu_limit",                       r"(?<=% CPULimit   : )\S+",                      int),
        ("pyres_version",                   r"(?<=% Version:  )(\S+)",                       str),
        ("rel_distance",                    r"(?<=# rel_distance: )\S+",                     int),
        ("graph_construction_time",         r"(?<=# graph_construction_time: )(\S+)",        float),
        ("neighbourhood_computation_time",  r"(?<=# neighbourhood_computation_time: )(\S+)", float),
        ("szs_status",                      r"(?<=% SZS status )(\S+)",                      str),
        ("initial_clauses",                 r"(?<=% Initial clauses    : )(\S+)",            int),
        ("processed_clauses",               r"(?<=% Processed clauses  : )(\S+)", 			 int),
        ("factors_computed",                r"(?<=% Factors computed   : )(\S+)", 			 int),
        ("resolvents_computed",             r"(?<=% Resolvents computed: )(\S+)", 			 int),
        ("tautologies_deleted",             r"(?<=% Tautologies deleted: )(\S+)", 			 int),
        ("forward_subsumed",                r"(?<=% Forward subsumed   : )(\S+)", 			 int),
        ("backward_subsumed",               r"(?<=% Backward subsumed  : )(\S+)", 			 int),
        ("user_time",                       r"(?<=% User time          : )(\S+)", 			 float),
        ("system_time",                     r"(?<=% System time        : )(\S+)", 			 float),
        ("total_time",                      r"(?<=% Total time         : )(\S+)", 			 float),
    ]

    for new_column_name, pattern, col_type in tqdm(new_column_specs, desc="Searching for regex patterns"):
        df_se_indi[new_column_name] = (
            df_se_indi["text"]
            .apply(lambda text: re.search(pattern + r"|$", text)[0])
            .apply(lambda text: col_type(text) if text !="" else None)
        )
    df_se_indi = df_se_indi.drop(["text"], axis="columns")

if store:
    df_se_indi.to_csv(problem_folder + f"df_se_indi_{str(job_index)}.csv")

df_se_indi

Searching in data/starexec_outputs/Job5201_output/


Searching for regex patterns: 100%|██████████| 18/18 [00:53<00:00,  2.98s/it]


,file_dir_name,problem,python_version,cpu_limit,pyres_version,rel_distance,graph_construction_time,neighbourhood_computation_time,szs_status,initial_clauses,processed_clauses,factors_computed,resolvents_computed,tautologies_deleted,forward_subsumed,backward_subsumed,user_time,system_time,total_time
0,data/starexec_outputs/Job5201_output/jannis_ge...,TOP008-1,3.13.0,5,1.5,21.0,0.220485,0.027002,ResourceOut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,data/starexec_outputs/Job5201_output/jannis_ge...,TOP009-1,3.13.0,5,1.5,21.0,0.218539,0.024398,ResourceOut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,data/starexec_outputs/Job5201_output/jannis_ge...,TOP019-1,3.13.0,5,1.5,21.0,0.208536,0.023816,ResourceOut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,data/starexec_outputs/Job5201_output/jannis_ge...,TOP018-1,3.13.0,5,1.5,21.0,0.211340,0.027290,ResourceOut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,data/starexec_outputs/Job5201_output/jannis_ge...,TOP001-2,3.13.0,5,1.5,21.0,0.002140,0.000724,ResourceOut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33323,data/starexec_outputs/Job5201_output/jannis_ge...,PUZ001-1,3.13.0,5,1.5,5.0,0.001429,0.000135,Unsatisfiable,12.0,23.0,0.0,18.0,0.0,5.0,6.0,0.274,0.024,0.298
33324,data/starexec_outputs/Job5201_output/jannis_ge...,PUZ001-1,3.13.0,5,1.5,33.0,0.001388,0.000816,Unsatisfiable,12.0,23.0,0.0,17.0,0.0,5.0,6.0,0.253,0.026,0.279
33325,data/starexec_outputs/Job5201_output/jannis_ge...,PUZ001-1,3.13.0,5,1.5,13.0,0.001403,0.000328,Unsatisfiable,12.0,23.0,0.0,18.0,0.0,5.0,6.0,0.265,0.027,0.292
33326,data/starexec_outputs/Job5201_output/jannis_ge...,PUZ001-1,3.13.0,5,1.5,54.0,0.001372,0.001361,Unsatisfiable,12.0,23.0,0.0,17.0,0.0,5.0,6.0,0.256,0.032,0.288


In [17]:

# pd.DataFrame([["foo","bar","foo","foo","foo","foo","bar","fbr"],["foo","bar","foo","foo","foo","foo","bar","fbr"],["foo","bar","foo","foo","foo","foo","bar","fbr"]], columns=["Hello", "Hello1",  "Hello1", "hello",  "Hello1",  "Hello1",  "Hello1",  "Hello1"]).to_xml("hello.xml")


pd.read_xml("test_xml_download.xml")

,name,JobAttributes,JobPair
0,test_xml_download,\n,NaN
